# 자연처리 프레임워크 및 라이브러리 설치하기

In [1]:
#  한국어 자연어 처리를 위한 형태소 분석기 패키지
!pip install konlpy

     |████████████████████████████████| 19.4MB 3.7MB/s 
     |████████████████████████████████| 92kB 12.1MB/s 
     |████████████████████████████████| 450kB 41.7MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [13]:
# 필요 프레임워크 및 라이브러리 준비
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re
import urllib.request
import json
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# 감성 분석
# 전처리 및 모델 로드

In [22]:
# 전처리 및 모델 로드
# 모델 가중치 파일 로드
loaded_model = load_model('sentiment_model_0.1.h5') # 감성 분류

# 토크나이징
okt = Okt() # 형태소 분석기 객체 생성
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다'] # 불용어 정의
max_len = 30 # 패딩 길이 정의
tokenizer = Tokenizer() # 토크나이저 객체 생성
with open('sentiment_vocab_0.1.json') as json_file:
  token = json.load(json_file)
  tokenizer.word_index = token

30 <class 'int'>


# 감성 분석
# 모델 실행 함수

In [33]:
# 학습한 모델에 새로운 예측값을 얻는 것은 model.predict() 사용
# 그리고 예측할 때에도 동일한 전처리 수행해야함.
# 감성분석 모델 실행 함수 
def sentiment_predict(new_sentence):
  new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
  new_sentence = okt.morphs(new_sentence, stem=True) # 토큰화,   "stem =True" 어간 추출(ex 해야지 -> 하다)
  new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
  encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
  pad_new = pad_sequences(encoded,maxlen = max_len) # 패딩
  print(encoded)
  print(maxlen)
  print(pad_new)
  score = float(loaded_model.predict(pad_new)) # 예측
  if(score > 0.5):
    print("{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score * 100))
  else:
    print("{:.2f}% 확률로 부정 리뷰입니다.\n".format((1 - score) * 100))

In [35]:
sentiment_predict('우선 영화는 먼저 스토리가 좋아야 합니다.거기에.영상미, 연기력까지 더해지면 몰입해서 볼 수 있는 재미난 영화가 탄생하게 되죠. 이 영화는 소재는 잘 끌어왔습니다. 근데 이게 다 입니다. 좋은 소재로 스토리? 정말 개연성 없구요. 영화가 이어질때 마다 A와 B 상황이 이어지기 위해 억지로 끼워 맞춰서몰입감이 떨어져버립니다. 그 상황에서 감동 연출? 몰입이 안되는 상황에서 어떤 연출이 효과를 발휘 하겠습니까? 그리고 관람객분들 위에도 말씀드렸다싶이 영화는 몰입할 수 있고 빠져들게 만들어여 합니다.(간추려서 재밌어야 합니다.) 소재가 좋았느니 연기력이 좋았다느니 영화가 재미없으면 다 말짱꽝이라구요. 이런영화 혹평을 해서 감독이 정신차리게끔 하던지 아니지면 사장되게 해서 더 재밌는 영화가 개봉할 수 있게 만들어야죠 이런 영화에 8점, 9점... 정말로 재밌으셨습니까?? ')

[[2540, 1, 1008, 33, 7, 167, 807, 255, 60, 1227, 292, 82, 68, 6, 56, 128, 1, 1929, 11, 1, 168, 63, 1009, 87, 267, 38, 9, 5, 7, 168, 17, 33, 10, 505, 140, 4, 1, 1296, 48, 383, 87, 543, 1296, 363, 936, 2069, 682, 292, 313, 300, 137, 29, 543, 21, 39, 110, 292, 97, 543, 21, 175, 110, 1064, 4477, 118, 3298, 583, 305, 346, 11564, 1, 292, 68, 6, 962, 22, 1156, 12, 168, 7, 255, 7, 1, 66, 9, 33893, 1647, 637, 130, 1, 3538, 3, 51, 4600, 18, 3246, 1145, 46, 12, 1, 261, 68, 6, 22, 42, 1, 20, 20, 1180, 12]]
[[  255     7     1    66     9 33893  1647   637   130     1  3538     3
     51  4600    18  3246  1145    46    12     1   261    68     6    22
     42     1    20    20  1180    12]]


InvalidArgumentError: ignored

In [36]:
sentiment_predict('이 영화 개꿀잼 ㅋㅋㅋ')

[[1, 3140, 92]]
[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    1
  3140   92]]
97.50% 확률로 긍정 리뷰입니다.



In [ ]:
sentiment_predict('이 영화 핵노잼 ㅠㅠ')

99.16% 확률로 부정 리뷰입니다.



In [ ]:
sentiment_predict('이딴게 영화냐 ㅉㅉ')

99.93% 확률로 부정 리뷰입니다.



In [ ]:
sentiment_predict('감독 뭐하는 놈이냐?')

99.08% 확률로 부정 리뷰입니다.



In [ ]:
sentiment_predict('감독 뭐하시는 분인지?')

98.12% 확률로 부정 리뷰입니다.



In [ ]:
sentiment_predict('대박 감독 뭐하시는 분?')

60.38% 확률로 부정 리뷰입니다.



In [ ]:
sentiment_predict('와 개쩐다 정말 세계관 최강자들의 영화다')

88.99% 확률로 긍정 리뷰입니다.



# 비속어 분류
# 전처리 및 모델 로드

In [8]:
# 전처리 및 모델 로드
# 모델 가중치 파일 로드
loaded_model = load_model('slang_model_0.1.h5') # 감성 분류

# 토크나이징
okt = Okt() # 형태소 분석기 객체 생성
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다'] # 불용어 정의
max_len = 30 # 패딩 길이 정의
tokenizer = Tokenizer() # 토크나이저 객체 생성
with open('slang_vocab_0.1.json') as json_file:
  token = json.load(json_file)
  tokenizer.word_index = token

# 비속어 분류
# 모델 실행 함수

In [9]:
# 학습한 모델에 새로운 예측값을 얻는 것은 model.predict() 사용
# 그리고 예측할 때에도 동일한 전처리 수행해야함.
# 감성분석 모델 실행 함수 
def slang_predict(new_sentence):
  new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
  new_sentence = okt.morphs(new_sentence, stem=True) # 토큰화,   "stem =True" 어간 추출(ex 해야지 -> 하다)
  new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
  encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
  pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
  score = float(loaded_model.predict(pad_new)) # 예측
  if(score > 0.5):
    print("{:.2f}% 확률로 비속어 포함입니다.\n".format(score * 100))
  else:
    print("{:.2f}% 확률로 비속어 미포함입니다.\n".format((1 - score) * 100))

In [11]:
slang_predict('우선 영화는 먼저 스토리가 좋아야 합니다.거기에.영상미, 연기력까지 더해지면 몰입해서 볼 수 있는 재미난 영화가 탄생하게 되죠. 이 영화는 소재는 잘 끌어왔습니다. 근데 이게 다 입니다. 좋은 소재로 스토리? 정말 개연성 없구요. 영화가 이어질때 마다 A와 B 상황이 이어지기 위해 억지로 끼워 맞춰서몰입감이 떨어져버립니다. 그 상황에서 감동 연출? 몰입이 안되는 상황에서 어떤 연출이 효과를 발휘 하겠습니까? 그리고 관람객분들 위에도 말씀드렸다싶이 영화는 몰입할 수 있고 빠져들게 만들어여 합니다.(간추려서 재밌어야 합니다.) 소재가 좋았느니 연기력이 좋았다느니 영화가 재미없으면 다 말짱꽝이라구요. 이런영화 혹평을 해서 감독이 정신차리게끔 하던지 아니지면 사장되게 해서 더 재밌는 영화가 개봉할 수 있게 만들어야죠 이런 영화에 8점, 9점... 정말로 재밌으셨습니까?? ')

InvalidArgumentError: ignored

In [10]:
slang_predict('개 새끼 씨발')

99.73% 확률로 비속어 포함입니다.



In [ ]:
slang_predict('애미는 있냐? 개같은 놈이네')
slang_predict('니 창자로 젓갈 담궈 버린다')
slang_predict('눈깔의 위치를 바꿔줘?')


83.72% 확률로 비속어 포함입니다.

57.13% 확률로 비속어 미포함입니다.

65.21% 확률로 비속어 미포함입니다.



In [ ]:
slang_predict('ㅅ1바라 뒤질래?')
slang_predict('애미 뒤진 새끼')
slang_predict('밥쳐먹고 그것만 하는 새끼가')
slang_predict('존나 못하네 진짜 쳐 나가라')

65.64% 확률로 비속어 미포함입니다.

98.78% 확률로 비속어 포함입니다.

93.77% 확률로 비속어 포함입니다.

93.99% 확률로 비속어 포함입니다.



97.50% 확률로 긍정 리뷰입니다.



94.23% 확률로 긍정 리뷰입니다.



97.65% 확률로 긍정 리뷰입니다.



87.07% 확률로 부정 리뷰입니다.



# 비속어

In [ ]:
slang_predict('개 새끼 씨발')

77.48% 확률로 비속어가 포함되지 않은 문장입니다..



In [ ]:
sentiment_predict('애미는 있냐? 개같은 놈이네')
sentiment_predict('니 창자로 젓갈 담궈 버린다')
sentiment_predict('눈깔의 위치를 바꿔줘?')
sentiment_predict('ㅅ1바라 뒤질래?')
sentiment_predict('애미 뒤진 새끼')
sentiment_predict('밥쳐먹고 그것만 하는 새끼가')
sentiment_predict('존나 못하네 진짜 쳐 나가라')

95.43% 확률로 부정 리뷰입니다.

96.85% 확률로 부정 리뷰입니다.

99.57% 확률로 부정 리뷰입니다.

62.19% 확률로 긍정 리뷰입니다.

92.32% 확률로 긍정 리뷰입니다.

87.35% 확률로 긍정 리뷰입니다.

99.33% 확률로 부정 리뷰입니다.

